In [1]:
from elasticsearch import Elasticsearch

# Create an Elasticsearch client instance with authentication
es = Elasticsearch(
    "https://localhost:9200",  # Use https for SSL
    basic_auth=(
        "elastic",
        "8+6x+mXMgyoyi0F7Fi07",
    ),  # Replace with your username and password
    verify_certs=False,  # Optional: Disable SSL certificate verification for development
    ssl_show_warn=False,  # This will disable the warning
)

format data as yaml

In [13]:
import yaml

# Define the search query
query = {"query": {"match": {"title": "pasta"}}}

# Execute the search query and retrieve the response
response = es.search(index="recipes", body=query)

# Convert the JSON response to YAML format and print it
yaml_response = yaml.dump(response, default_flow_style=False)
print(yaml_response)

!!python/object/new:elastic_transport.ObjectApiResponse
state: !!python/tuple
- _shards:
    failed: 0
    skipped: 0
    successful: 1
    total: 1
  hits:
    hits: []
    max_score: null
    total:
      relation: eq
      value: 0
  timed_out: false
  took: 4
- !!python/object:elastic_transport.ApiResponseMeta
  duration: 0.018866539001464844
  headers: !!python/object/new:elastic_transport.HttpHeaders
    state: !!python/tuple
    - null
    - _frozen: false
      _internal:
        content-type: !!python/tuple
        - content-type
        - application/vnd.elasticsearch+json;compatible-with=8
        transfer-encoding: !!python/tuple
        - Transfer-Encoding
        - chunked
        x-elastic-product: !!python/tuple
        - X-elastic-product
        - Elasticsearch
  http_version: '1.1'
  node: !!python/object:elastic_transport.NodeConfig
    _extras: {}
    ca_certs: null
    client_cert: null
    client_key: null
    connections_per_node: 10
    headers: !!python/object

format data as json

In [17]:
import json

# Define the search query
query = {"query": {"match": {"title": "pasta"}}}

# Execute the search query with pretty formatting enabled
response = es.search(index="recipes", body=query)

# Convert the JSON response to pretty JSON format and print it
pretty_json_response = json.dumps(response.body, indent=4)
print("Pretty JSON Response:")
print(pretty_json_response)

Pretty JSON Response:
{
    "took": 5,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 0,
            "relation": "eq"
        },
        "max_score": null,
        "hits": []
    }
}


source filtering

In [18]:
# Excluding the _source field altogether
query1 = {"_source": False, "query": {"match": {"title": "pasta"}}}
response1 = es.search(index="recipes", body=query1)
print(response1)

# Only returning the 'created' field
query2 = {"_source": "created", "query": {"match": {"title": "pasta"}}}
response2 = es.search(index="recipes", body=query2)
print(response2)

# Only returning an object's key (e.g., ingredients.name)
query3 = {"_source": "ingredients.name", "query": {"match": {"title": "pasta"}}}
response3 = es.search(index="recipes", body=query3)
print(response3)

# Returning all of an object's keys (e.g., ingredients.*)
query4 = {"_source": "ingredients.*", "query": {"match": {"title": "pasta"}}}
response4 = es.search(index="recipes", body=query4)
print(response4)

# Returning the ingredients object with all keys, and the servings field
query5 = {
    "_source": ["ingredients.*", "servings"],
    "query": {"match": {"title": "pasta"}},
}
response5 = es.search(index="recipes", body=query5)
print(response5)

# Including all of the ingredients object's keys, except the name key
query6 = {
    "_source": {"includes": "ingredients.*", "excludes": "ingredients.name"},
    "query": {"match": {"title": "pasta"}},
}
response6 = es.search(index="recipes", body=query6)
print(response6)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0

controlling the size

In [19]:
# 1. Using the `size` parameter as a query parameter
response1 = es.search(
    index="recipes",
    body={"_source": False, "query": {"match": {"title": "pasta"}}},
    params={"size": 2},  # size as a query parameter
)
print(response1)

# 2. Including `size` within the request body
response2 = es.search(
    index="recipes",
    body={
        "_source": False,
        "size": 2,  # size within the request body
        "query": {"match": {"title": "pasta"}},
    },
)
print(response2)

{'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


/tmp/ipykernel_8284/122332058.py:2: DeprecationWarning: The 'params' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response1 = es.search(
/tmp/ipykernel_8284/122332058.py:2: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response1 = es.search(


offset

In [20]:
# Using the `from` parameter to specify the offset
response = es.search(
    index="recipes",
    body={
        "_source": False,  # Exclude _source fields from the response
        "size": 2,  # Number of results to return
        "from": 2,  # Offset to start from (skip the first 2 results)
        "query": {"match": {"title": "pasta"}},
    },
)

# Print the response to verify the result
print(response)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


sorting

In [21]:
# Sorting by ascending order (implicitly)
response_ascending = es.search(
    index="recipes",
    body={
        "_source": False,
        "query": {"match_all": {}},
        "sort": ["preparation_time_minutes"],  # Ascending by default
    },
)
print("Ascending order response:", response_ascending)

# Sorting by descending order
response_descending = es.search(
    index="recipes",
    body={
        "_source": "created",
        "query": {"match_all": {}},
        "sort": [{"created": "desc"}],  # Sorts by "created" in descending order
    },
)
print("Descending order response:", response_descending)

# Sorting by multiple fields
response_multiple_sort = es.search(
    index="recipes",
    body={
        "_source": ["preparation_time_minutes", "created"],
        "query": {"match_all": {}},
        "sort": [
            {"preparation_time_minutes": "asc"},  # First sort field in ascending order
            {"created": "desc"},  # Second sort field in descending order
        ],
    },
)
print("Multiple fields sorted response:", response_multiple_sort)

Ascending order response: {'took': 68, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
Descending order response: {'took': 21, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
Multiple fields sorted response: {'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


multi value field

In [22]:
# Sorting by the average rating in descending order
response_sort_by_avg_rating = es.search(
    index="recipes",
    body={
        "_source": "ratings",  # Only returning the "ratings" field
        "query": {"match_all": {}},
        "sort": [
            {
                "ratings": {
                    "order": "desc",  # Descending order
                    "mode": "avg",  # Sorting by the average value
                }
            }
        ],
    },
)

# Output the results
print("Sorted by average rating (descending):", response_sort_by_avg_rating)

Sorted by average rating (descending): {'took': 27, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
